<a href="https://colab.research.google.com/github/Benendead/LSTMjazz/blob/master/Research/Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
Author(s) | Year | Models Used | Task | Useful lessons | Future work
--- | ---


**Topics Left To Study** (Paper Mentioned In)  
* RTRL error (Eck 2002)
* BPTT (Eck 2002)
* Kalman filter to update weights? (Eck 2002)
* [RBM](http://imonad.com/rbm/restricted-boltzmann-machine/) (Bickerman 2010)
* [Contrastive Divergence](http://www.cs.toronto.edu/~fritz/absps/tr00-004.pdf) (Bickerman 2010)

# Summaries

## 1. Choi et al. (2016)

**Summary**  
They used char-RNN and word-RNN to generate jazz chord progressions. Their data was 2,486 scores transposed to C from real books. The models were fed text data of chord progressions, with all quarter notes filled in, as in:  
-START- F:9 F:9 F:9 F:9 D:min7 D:min7 G:9 G:9  
C:maj C:maj F:9 F:9 C:maj C:maj C:maj C:maj -END-  
There were 39 distinct characters, 1,259 distinct chords, and 539,609 chords overall.

**Why char-RNN?**  
Benefits of char-RNN:
* Smaller vocabulary
* Fewer assumptions about music made

Downsides of char-RNN:
* Shorter effective length of memory, as more time steps are needed to input the same information

**Model Architecture**  
Two LSTM layers with 512 hidden units (hidden state dimensionality) and then Dropout of 0.2 after each LSTM layer. Put together in Keras with categorical cross entropy as loss, Adam optimizer, and stochastic prediction based on a diversity parameter $\alpha$. New probabilities are calculated:  
$\hat{p}_i=e^{log(p_i)/\alpha}$, where $p_i$ is the probability for the $i$ states.  
A state is then selected based on the probabilities.

**Successes**  
Their char-RNN learned $\text{ii}-\text{V}-\text{I}$, passing chords, modal interchange chords, and tritone substitutions. The word-RNN included modal interchange, circle of fifths progressions, and descending basslines. Their code is available [here](https://github.com/keunwoochoi/lstm_real_book).  
*-Fin 1/24/2019-*

## 2. Eck et al. (2002)

**Summary**  
This paper ran two experiments with LSTMs:
1. Memorizing a chord progression.
2. Also improvising pentatonic blues over a memorized progression.  

Both experiments were successful using relatively simple LSTMs. They represented the data as a vector with 12 possible chord notes and 13 melody notes. Their quantization step was an eighth note and note representation was a simple 1/0 on/off input.  

Examples can be found [here](https://web.archive.org/web/20190104192500/http://people.idsia.ch/~juergen/blues/).

**Experiment 1**  
The data was just the same progression repeatedly. The network architecture used 4 LSTM cells each with 2 hidden cells all fully connected to each other and the input layer. Forget, input, and output gate biases were set to -0.5, -1.0, -1.5, and -2.0 for the four blocks, respectively. Output biases were 0.5, learning rate 0.00001, momentum rate 0.9. Loss was cross-entropy and output function was logistic sigmoid.

In the output, chord notes were on if they had a probability above 0.5. Training ended once the chord sequence was completely memorized.

**Experiment 2**  
Some cell blocks learned melody and others learned chords. There were eight cell blocks total with two hidden cells each. Melody information never reached chord cell blocks, although everything else was identical to Experiment 1.

Again, chord outputs had a 0.5 threshold. The melody output needed to pick one note, so the probabilities were constrained to sum to 1 and then a random number in [0,1] was used. Training ended until objective error plateaued. Outputs were seeded and then allowed to compose progressions.  
*-Fin 10/24/2019-*

## 3. Bickerman et al. (2010)

**Summary**  
I was into the data representation part of rereading my past notes.